In [1]:
import requests
import datetime
import re
import pandas as pd
import pyspark
from pyspark.sql import functions as f, SparkSession,SQLContext
from pyspark.sql.types import StructType, StructField, StringType

sc = pyspark.SparkContext('local[*]')
spark      =  SparkSession(sc)

In [2]:
def return_yyyymm(starting_yyyy,ending_yyyy):
    return [(int((str(i) + format(j, '02d')))) for i in range(starting_yyyy,ending_yyyy) for j in range(1,13)]

In [4]:
def extract_weather(lst_yyyymm,lst_stations):
    weather_header=  [
              "date",
              "minimum_temperature",
              "maximum_temperature",
              "rainfall_mm",
              "evaporation_mm",
              "sunshine_hours",
              "direction_of_maximum_wind_gust",
              "speed_of_maximum_wind_gust_kmh",
              "time_of_maximum_wind_gust",
              "9am_temperature",
              "9am_relative_humidity",
              "9am_cloud_amount_oktas",
              "9am_wind_direction",
              "9am_wind_speed_kmh",
              "9am_msl_pressure_hpa",
              "3pm_temperature",
              "3pm_relative_humidity",
              "3pm_cloud_amount_oktas",
              "3pm_wind_direction",
              "3pm_wind_speed_kmh",
              "3pm_msl_pressure_hpa"]
    weather_header_1 = ["Empty"] + [i for i in weather_header]
    list_pd_obs =  []    
    for stations_no in lst_stations:
        # Run the loop for year - 12 months
        for yyyymm in lst_yyyymm:
            fcsv ="http://www.bom.gov.au/climate/dwo/{0}/text/IDCJDW{1}.{0}.csv".format(yyyymm,stations_no)
            #print(fcsv)
            responsed = requests.get(fcsv)      
            lst_responsed = responsed.content.decode('windows-1252').strip().split("\n")    
            if ([tup[0] for tup in  enumerate(lst_responsed) if '"Date",' in tup[1]]):
                
                header_position          =  ([tup[0] for tup in  enumerate(lst_responsed) if '"Date",' in tup[1]])[0]
                #print(header_position)
                list_data = lst_responsed[header_position+1:]
                lst_station_id_name = lst_responsed[header_position-2].split("from")[1].replace('"\r',"").replace("}","").split("{station ")
                station_name = lst_station_id_name[0] 
                station_id   = lst_station_id_name[1] 
                pd_weather   = pd.DataFrame([i.replace("\r","").split(",") for i in list_data],columns=weather_header_1)    
                pd_weather["station_name"]  = station_name
                pd_weather["station_id"]    = station_id
                list_pd_obs.append(pd_weather)
            else:
                print("No data for the period {0}".format(yyyymm))
    pd_all = pd.concat(list_pd_obs)
    pd_all["station_id"] = pd_all["station_id"].fillna(0.0).astype(int)
    
    # Import Weather Master data which has details about the weather station - join by station_id
    weather_master = pd.read_csv("/home/jovyan/data/all_station_master.csv")
    weather_master["station_id"] = weather_master["Site"].fillna(0.0).astype(int)
    
    # join the weather master table using station id  
    new_df = pd.merge(pd_all, weather_master,  how='inner', left_on=['station_id'], right_on = ['station_id'])
    df_bom = new_df.drop('Empty', 1)
    
    # Save the PD data frame using Spark as csv file with valid columns
    df_bom_schema   = StructType([StructField(col,StringType(),True) for col in df_bom.columns])
    df_bom_obs  = spark.createDataFrame(df_bom,schema=df_bom_schema)
    df_bom_obs.write.csv('data/aus_weather_extract.csv',mode="overwrite")

In [5]:
lst_stations = [2801,2802,2124,2111,2098,2146,3033,3005,2002,3008,4019,4024,4128,4050,5002,5041,5059,5055,6111,6001,6061,6015]
extract_weather(return_yyyymm(2017,2018),lst_stations) 

No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
No data for the period 201701
